# YOLO を使った物体検出

## インストール

In [ ]:
!pip install -r tensorflow-yolov4-tflite/requirements.txt

## 定数

In [ ]:
# 適当な画像を用意する
IMAGE_PATH = "dog.jpg"

# 入力データのサイズ
INPUT_SIZE = 416

# 推論結果のスコアに対するしきい値
SCORE_THRESHOLD = 0.5

# Non-Maximun Suppression における IoU のしきい値
IOU_THRESHOLD = 0.5

## インポート

In [ ]:
import os
from pathlib import Path

import tensorflow as tf
import numpy as np
import cv2 as cv
from PIL import Image, ImageDraw, ImageColor, ImageFont
import certifi

## モデルのダウンロード

In [ ]:
# ダウンロードに失敗しないようにするためのおまじない
os.environ["SSL_CERT_FILE"] = certifi.where()

In [ ]:
tf.keras.utils.get_file(
    fname="yolov4.cfg",
    origin="https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.cfg",
    cache_subdir="models/yolov4",
)
model_dir = tf.keras.utils.get_file(
    fname="yolov4.weights",
    origin="https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights",
    cache_subdir="models/yolov4",
)
model_dir = str(Path(model_dir).parent)
model_dir

## モデルを SavedModel に変換

In [ ]:
!cd tensorflow-yolov4-tflite && python save_model.py --weights $model_dir/yolov4.weights --output $model_dir/saved_model --input_size $INPUT_SIZE --model yolov4

## モデルをロード

In [ ]:
model = tf.saved_model.load(str(Path(model_dir).joinpath("saved_model")))
infer = model.signatures["serving_default"]

!saved_model_cli show --all --dir $model_dir/saved_model

In [ ]:
with Path(model_dir).joinpath("coco.names").open() as f:
    category_index = {id: {"id": id, "name": name.strip()} for id, name in enumerate(f)}

## 画像データをロード

In [ ]:
image = tf.io.read_file(IMAGE_PATH)
image = tf.io.decode_jpeg(image)

## 画像データを正規化

In [ ]:
images = tf.image.convert_image_dtype(image, tf.float32)
images = tf.image.resize(images, (INPUT_SIZE, INPUT_SIZE))
images = tf.stack([images])
images

## 推論を実行

In [ ]:
preds = infer(images)

boxes = preds["tf_op_layer_concat_18"][:, :, :4]
boxes = tf.stack([boxes], axis=2)

scores = preds["tf_op_layer_concat_18"][:, :, 4:]

boxes.shape, scores.shape

## Non-Maximun Suppression

In [ ]:
boxes, scores, labels, num_detections = tf.image.combined_non_max_suppression(
    boxes,
    scores,
    max_output_size_per_class=50,
    max_total_size=50,
    score_threshold=SCORE_THRESHOLD,
    iou_threshold=IOU_THRESHOLD,
)

boxes = boxes[0]
scores = scores[0]
labels = labels[0]
num_detections = int(num_detections[0])

num_detections, boxes.shape, scores.shape, labels.shape

## バウンディングボックスを描画

In [ ]:
output_image = Image.open(IMAGE_PATH)

draw = ImageDraw.Draw(output_image)
color = ImageColor.getrgb("#E91E63")
font = ImageFont.truetype("Monaco", size=46)

xy_scale = np.array([output_image.height, output_image.width, output_image.height, output_image.width])

for i in range(num_detections):
    label = int(labels[i])
    score = scores[i]

    if score <= SCORE_THRESHOLD:
        continue

    name = category_index[label]["name"]
    y_min, x_min, y_max, x_max = boxes[i] * xy_scale

    draw.text((x_min, y_min), name, font=font, fill=color)
    draw.rectangle((x_min, y_min, x_max, y_max), outline=color, width=4)

output_image